This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '3ZVRU8CNbXqCCfYu9wDr'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2020-06-07&end_date=2020-06-08&api_key="+API_KEY
response = requests.get(url)
print(response.status_code)

200


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
import json

print(response.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2020-12-01T14:48:09.907Z', 'newest_available_date': '2020-12-01', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2020-06-07', 'end_date': '2020-06-08', 'data': [['2020-06-08', 92.65, 92.65, 89.8, 91.7, None, 172403.0, 15698539.85, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
# Collect data from FSE for the ticker AFX_X for the whole year 2017
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?&start_date=2017-01-01&end_date=2017-12-31&api_key="+API_KEY
response = requests.get(url)
print(response.status_code)

200


In [8]:
# Convert the returned json into a Python dictionary
response_json = response.json()
print(type(response_json))
print(response_json['dataset']['data'][0])

<class 'dict'>
['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None]


In [9]:
# Calculate the highest and lowest opening prices for the stock in this period
# Open: Index 1
data = response_json['dataset']['data']

highest_open_price = data[0][1]
lowest_open_price = data[0][1]

for daily_data in data:
    open_price = daily_data[1]
    if open_price: # null check
        if (open_price > highest_open_price):
            highest_open_price = open_price
        if (open_price < lowest_open_price):
            lowest_open_price = open_price

print("Highest opening price: " + str(highest_open_price))
print("Lowest opening price: " + str(lowest_open_price))

Highest opening price: 53.11
Lowest opening price: 34.0


In [13]:
# What was the largest change in any one day?
# High: Index 2, Low: Index 3
max_price_delta = 0.0

for daily_data in data:
    if (daily_data[2] and daily_data[3]): # null check
        price_delta = daily_data[2] - daily_data[3]
        if (price_delta > max_price_delta):
            max_price_delta = price_delta

print("Largest change in any one day: " + "{:.2f}".format(max_price_delta))

Largest change in any one day: 2.81


In [15]:
# What was the largest change between any two days?
# Close: Index 4
max_close_delta = 0.0

for index in range(len(data) - 1): # Avoid out of bounds error
    close_price = data[index][4]
    next_day_close_price = data[index+1][4]
    if (close_price and next_day_close_price): # null check
        close_delta = abs(next_day_close_price - close_price)
        if (close_delta > max_close_delta):
            max_close_delta = close_delta

print("Largest change between two days: " + "{:.2f}".format(max_close_delta))

Largest change between two days: 2.56


In [17]:
# What was the average daily trading volume during this year?
# Traded Volume: Index 6
traded_volumes = []

for daily_data in data:
    traded_volume = daily_data[6]
    if (traded_volume): # null check
        traded_volumes.append(traded_volume)

print("Number of entries: " + str(len(traded_volumes)))
print("Average daily trading volume: " + "{:.2f}".format(sum(traded_volumes)/len(traded_volumes)))

Number of entries: 255
Average daily trading volume: 89124.34


In [20]:
def median(list):
    sorted_list = sorted(list)
    length = len(list)
    if (length == 0 or length % 2 == 1):
        return sorted_list[length//2]
    else:
        return (sorted_list[length//2] + sorted_list[length//2 + 1])/2

In [21]:
# What was the median trading volume during this year?
# Traded Volume: Index 6
print("Median trading volume: " + str(median(traded_volumes)))

Median trading volume: 76286.0
